In [12]:
import os
import pandas as pd
import pickle
import requests
import warnings
from copy import deepcopy as dc
from random import choice
from skops.io import load

warnings.filterwarnings("ignore")

DIR = "../data/cluster1/"
PATH_MODEL = os.path.join(DIR, "census_model.skops")
PATH_DATA = os.path.join(DIR, "census.csv")

model = load(PATH_MODEL, trusted=True)
df = pd.read_csv(PATH_DATA)

def get_data(df):
    data = {
        'age': [],
        'workclass': [],
        'education.num': [],
        'marital.status': [],
        'occupation': [],
        'relationship': [],
        'race': [],
        'sex': [],
        'capital.gain': [],
        'capital.loss': [],
        'hours.per.week': [],
        'native.country': []
    }
    mapped = {}
    ids = []
    incomes = []
    columns = ['id', 'age', 'workclass', 'education.num', 'marital.status', 'occupation', 'relationship', 'race', 'sex', 'capital.gain', 'capital.loss', 'hours.per.week', 'native.country', 'income']
    for row in df.iterrows():
        for key, value in zip(columns, row[1]):
            if key == "id":
                mapped[int(value)] = row[1]
                ids.append(int(value))
            elif key == "income":
                incomes.append(value)
            else:
                data[key].append(value)
    return pd.DataFrame(data), ids, incomes, columns, mapped

data, ids, incomes, columns, mapped = get_data(df)
res = model.predict(data)
sus = []
for i in range(len(ids)):
    id = ids[i]
    act = incomes[i]
    pred = res[i]
    if act != pred and act == "<=50K":
        sus.append(id)

_sus = []
for i in range(len(sus)):
    for key, value in zip(columns, mapped[sus[i]]):
        if key in ("id", "income"): continue
        if key == "occupation" and value == "Tech-support":
            _sus.append(sus[i])
sus = _sus

def query(input_data):
    return requests.post("http://cluster1.advml.com/score", json={"data": input_data}).json()

best = query(sus)["s"]
vals = dc(sus)
while True:
    tmp = dc(vals)
    rem = choice(vals)
    vals.remove(rem)
    res = query(vals)
    if "s" not in res:
        with open("cluster1.pkl", "wb+") as f:
            pickle.dump(vals, f)
        break
    if res["s"] > best:
        best = res["s"]
        print(best)
    else:
        vals = tmp

## Note

I didn't save the list of IDs that got me the flag during the competition and, since the endpoint is no longer available, I can't rerun the above solution. The following code just dumps a list of zeros to a Pickle file instead.

In [15]:
with open("cluster1.pkl", "wb+") as f:
    pickle.dump([0] * 2000, f)